In [1]:
import os
import socket
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, length, when, col
from pyspark.sql.types import BooleanType, IntegerType, LongType, StringType, ArrayType, FloatType, StructType, StructField
from pyspark.sql.functions import *
from pyspark.sql.functions import pandas_udf
from pyspark.sql.functions import PandasUDFType
from jinja2 import Environment, FileSystemLoader

In [2]:
# setting constants
APP_NAME = "LAB-2-ANTONOV-PRODUCER"
NORMALIZED_APP_NAME = APP_NAME.replace('/', '_').replace(':', '_')

APPS_TMP_DIR = os.path.join(os.getcwd(), "tmp")
APPS_CONF_DIR = os.path.join(os.getcwd(), "conf")
APPS_LOGS_DIR = os.path.join(os.getcwd(), "logs")
LOG4J_PROP_FILE = os.path.join(APPS_CONF_DIR, "pyspark-log4j-{}.properties".format(NORMALIZED_APP_NAME))
LOG_FILE = os.path.join(APPS_LOGS_DIR, 'pyspark-{}.log'.format(NORMALIZED_APP_NAME))
EXTRA_JAVA_OPTIONS = "-Dlog4j.configuration=file://{} -Dspark.hadoop.dfs.replication=1 -Dhttps.protocols=TLSv1.0,TLSv1.1,TLSv1.2,TLSv1.3"\
    .format(LOG4J_PROP_FILE)

LOCAL_IP = socket.gethostbyname(socket.gethostname())

# preparing configuration files from templates
for directory in [APPS_CONF_DIR, APPS_LOGS_DIR, APPS_TMP_DIR]:
    if not os.path.exists(directory):
        os.makedirs(directory)

env = Environment(loader=FileSystemLoader('/opt'))
template = env.get_template("pyspark_log4j.properties.template")
template\
    .stream(logfile=LOG_FILE)\
    .dump(LOG4J_PROP_FILE)

#running spark
SPARK_ADRESS = "local[4]"

# run spark
spark = SparkSession\
        .builder\
        .appName(APP_NAME)\
        .master(SPARK_ADRESS)\
        .config('spark.ui.port', "4040")\
        .config("spark.memory.fraction", "0.8")\
        .config("spark.memory.storageFraction", "0.6")\
        .config("spark.driver.memory", "4g")\
        .config("spark.driver.extraJavaOptions", EXTRA_JAVA_OPTIONS)\
        .config("spark.executor.memory", "6g")\
        .getOrCreate()


In [3]:
# printing important urls and pathes
print("Web UI: {}".format(spark.sparkContext.uiWebUrl))
print("\nlog4j file: {}".format(LOG4J_PROP_FILE))
print("\ndriver log file: {}".format(LOG_FILE))

Web UI: http://jupyter-spark-668478f5f7-k97x4:4040

log4j file: /home/jovyan/nfs-home/LABS/LAB_KAFKA/PROD/conf/pyspark-log4j-LAB-2-ANTONOV-PRODUCER.properties

driver log file: /home/jovyan/nfs-home/LABS/LAB_KAFKA/PROD/logs/pyspark-LAB-2-ANTONOV-PRODUCER.log


## Step 1
Read and prepare the daata

Lets read posts data, exluding posts with no text in them

In [4]:
posts_df = spark.read.json("hdfs:///shared/bigdata20/followers_posts_api_final.json")\
.where((col("text") != "") & (col("text") != " "))\
.select(col("owner_id").alias("user"), "date", "text")

posts_df.show()

+-----+----------+--------------------+
| user|      date|                text|
+-----+----------+--------------------+
|87449|1550165023|Я люблю Вас. Я вч...|
|87449|1553774858|call me by your n...|
|87449|1555602008|                  🦋|
|87449|1558105050|         Браво,Юра !|
|87449|1559301729|                  🕊|
|87449|1564088944|Самый неприятный ...|
|87449|1565162341|                  🔅|
|87449|1566293696|Пекло.Пушка.Вышка...|
|94494|1559647730|Тот неловкий моме...|
|94494|1559854126|Белые ночи + жарк...|
|94494|1560680534|Не велосипедом ед...|
|94494|1560987374|Тот самый француз...|
|94494|1561155082|Да, знатное шоу с...|
|94494|1562425403|На акциях #Раздел...|
|94494|1562433670|А ещё сегодня на ...|
|94494|1566767811|Говорят, давно от...|
|94494|1567495072|В этом году получ...|
|94494|1567620712|Не успели ещё под...|
|94494|1567951378|Ого, кажется в эт...|
|94494|1568279689|Здорово, что в эт...|
+-----+----------+--------------------+
only showing top 20 rows



Now lets read the data on followers. Lets exclude people with null sex and age, because we will not be able to assign them to sex and age groups later on

In [5]:
followers_df = spark.read.json("hdfs:///shared/bigdata20/followers_info.json")\
.select(col("id").alias("user"), "sex", "bdate")\
.withColumn("month_between", 
    months_between(
        current_date(), 
        concat_ws(
            "-", regexp_extract(col("bdate"), r"(\d{1,2})\.(\d{1,2})\.(\d{4})", 3).cast("int"), 
            regexp_extract(col("bdate"), r"(\d{1,2})\.(\d{1,2})\.(\d{4})", 2).cast("int"), 
            regexp_extract(col("bdate"), r"(\d{1,2})\.(\d{1,2})\.(\d{4})", 1).cast("int")
        ).cast("date")
    )
)\
.select("user", "sex", (col("month_between")/12).cast("int").alias("age"))\
.where(col("sex").isNotNull() & col("age").isNotNull())

followers_df.show()

+----+---+---+
|user|sex|age|
+----+---+---+
| 243|  1| 34|
| 533|  2| 32|
| 637|  1| 36|
| 751|  2| 33|
| 899|  1| 33|
|1087|  2| 36|
|1110|  2| 33|
|1113|  2| 33|
|1174|  1| 31|
|1390|  2| 31|
|1398|  2| 35|
|1598|  2| 33|
|1669|  1| 32|
|1737|  2| 32|
|1772|  2| 32|
|1822|  2| 34|
|2057|  1| 32|
|2192|  2| 33|
|2212|  2| 32|
|2461|  1| 38|
+----+---+---+
only showing top 20 rows



Now lets join info about users and posts

In [6]:
join_df = posts_df.join(followers_df, ["user"])\
.select(
    "user", 
    col("date").alias("date_posted"), 
    "text", 
    (when(col("sex") == 1, "M").otherwise("F")).alias("sex"),
    "age"
)

join_df.show()

+-----+-----------+--------------------+---+---+
| user|date_posted|                text|sex|age|
+-----+-----------+--------------------+---+---+
|94494| 1559647730|Тот неловкий моме...|  F| 33|
|94494| 1559854126|Белые ночи + жарк...|  F| 33|
|94494| 1560680534|Не велосипедом ед...|  F| 33|
|94494| 1560987374|Тот самый француз...|  F| 33|
|94494| 1561155082|Да, знатное шоу с...|  F| 33|
|94494| 1562425403|На акциях #Раздел...|  F| 33|
|94494| 1562433670|А ещё сегодня на ...|  F| 33|
|94494| 1566767811|Говорят, давно от...|  F| 33|
|94494| 1567495072|В этом году получ...|  F| 33|
|94494| 1567620712|Не успели ещё под...|  F| 33|
|94494| 1567951378|Ого, кажется в эт...|  F| 33|
|94494| 1568279689|Здорово, что в эт...|  F| 33|
|94494| 1568391025|Всё жду тот светл...|  F| 33|
|94494| 1568409427|Пешеходные перехо...|  F| 33|
|94494| 1568498829|Про деревья в гор...|  F| 33|
|94494| 1568543078|В огромном парке ...|  F| 33|
|94494| 1568559778|Рядом с железнодо...|  F| 33|
|94494| 1568582668|А

## Step 2
Creating a producer

In [ ]:
import time
import json
import random
from tqdm import tqdm
from kafka import KafkaProducer

producer = KafkaProducer(bootstrap_servers="kafka-svc:9092", value_serializer=str.encode)

for row in tqdm(join_df.orderBy("date_posted").rdd.toLocalIterator(), total=join_df.count()):
    d = row.asDict()
    print(f"Sending a post from user id: {d['user']}, posted on: {d['date_posted']}, sex - {d['sex']}")
    
    producer.send("main", json.dumps(d, ensure_ascii=False))
    producer.flush()
    time.sleep(random.uniform(0.1, 2.))
    
print("done")

  0%|          | 0/73624 [00:00<?, ?it/s]

Sending a post from user id: 273486249, posted on: 1546290007, sex - F


  0%|          | 1/73624 [00:13<279:20:45, 13.66s/it]

Sending a post from user id: 30697, posted on: 1546290009, sex - F


  0%|          | 2/73624 [00:15<133:01:12,  6.50s/it]

...

Sending a post from user id: 49730393, posted on: 1546377234, sex - M


  0%|          | 343/73624 [06:16<30:34:09,  1.50s/it]

Sending a post from user id: 7692544, posted on: 1546377623, sex - M


  0%|          | 344/73624 [06:18<31:45:26,  1.56s/it]

Sending a post from user id: 278435421, posted on: 1546377644, sex - F


In [ ]:
spark.stop()